# Production ready crosslight detection

### Lets create our model and load the weights

In [ ]:
import display
display.five()

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

from picamera import PiCamera, array
from time import sleep
from numpy import around

display.four()

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(112, 112, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
binary_crossentropy
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

display.three()

model.load_weights('/home/pi/repositories/pren/models/crosslight.h5')

display.two()

### Raspberry camera logic

In [ ]:
def checkPrediction(prediction):
    if prediction[1] > 0.95:
        display.zero()

    if prediction[0] > 0.95:
        print("go")
        break

In [ ]:
with PiCamera() as camera:
    camera.resolution = (112, 112)
    
    # Sleep to let camera adjust to the light
    sleep(1)
    
    display.one()
    with array.PiRGBArray(camera) as output:
        while True:
            camera.capture(output, 'rgb')
            # Lösungsweg 1
            img = output.array.astype(float)
            img = img.reshape((1,) + img.shape)
            prediction = around(model.predict(img), 3)[0]
            
            # Lösungsweg 2
            #prediction = around(model.predict(output.array), 3)[0]
            
            checkPrediction(prediction)
            output.truncate(0)